<a href="https://colab.research.google.com/github/starminalush/mlops_report/blob/main/tensorrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка TensorRT

Есть уже готовые [образы Docker](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tensorrt/tags), где все поставлено и работает. Лучше использовать их. Но если у вас из доступной карты с тензорными ядрами есть только Tesla K80 на колабе, то приведем пример установки TensorRT из [архива](https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html#installing-tar)

1. Узнаем версию CUDA на видеокарте

In [ ]:
!nvidia-smi

Thu May  5 19:43:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

2. Узнаем версию системы

In [ ]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


3. Основываясь на том, что у нас ubuntu18.04 и CUDA 11.6, найдем архив нужной нам версии TensorRT с сайта [NVIDIA](https://developer.nvidia.com/nvidia-tensorrt-8x-download), для нас это будет TensorRT-8.2.1.8. Скачаем его и поместим на google drive

4. Распакуем архив

In [ ]:
!tar -xzvf /content/drive/MyDrive/TensorRT-8.2.1.8.Linux.x86_64-gnu.cuda-11.4.cudnn8.2.tar.gz

TensorRT-8.2.1.8/
TensorRT-8.2.1.8/TensorRT-Release-Notes.pdf
TensorRT-8.2.1.8/onnx_graphsurgeon/
TensorRT-8.2.1.8/onnx_graphsurgeon/onnx_graphsurgeon-0.3.12-py2.py3-none-any.whl
TensorRT-8.2.1.8/samples/
TensorRT-8.2.1.8/samples/sampleNMT/
TensorRT-8.2.1.8/samples/sampleNMT/model/
TensorRT-8.2.1.8/samples/sampleNMT/model/likelihoodCombinationOperator.h
TensorRT-8.2.1.8/samples/sampleNMT/model/contextNMT.cpp
TensorRT-8.2.1.8/samples/sampleNMT/model/multiplicativeAlignment.h
TensorRT-8.2.1.8/samples/sampleNMT/model/slpAttention.h
TensorRT-8.2.1.8/samples/sampleNMT/model/decoder.h
TensorRT-8.2.1.8/samples/sampleNMT/model/lstmEncoder.cpp
TensorRT-8.2.1.8/samples/sampleNMT/model/beamSearchPolicy.h
TensorRT-8.2.1.8/samples/sampleNMT/model/beamSearchPolicy.cpp
TensorRT-8.2.1.8/samples/sampleNMT/model/componentWeights.h
TensorRT-8.2.1.8/samples/sampleNMT/model/embedder.h
TensorRT-8.2.1.8/samples/sampleNMT/model/encoder.h
TensorRT-8.2.1.8/samples/sampleNMT/model/softmaxLikelihood.h
TensorRT-8.

5. Установим TensorRT в соответствии с [мануалом](https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html#installing-tar) и версией питона

In [ ]:
!python --version

Python 3.7.13


In [ ]:
!python3 -m pip install TensorRT-8.2.1.8/python/tensorrt-8.2.1.8-cp37-none-linux_x86_64.whl

Processing ./TensorRT-8.2.1.8/python/tensorrt-8.2.1.8-cp37-none-linux_x86_64.whl


In [ ]:
!pip install pycuda pytools==2022.1.1

  Using cached pycuda-2021.1.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 67 kB 3.1 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
  Created wheel for pytools: filename=pytools-2022.1.1-py2.py3-none-any.whl size=63859 sha256=11eddc872b09065c8590c4ab17d71bf7c7acd4bd8329140fc3405907716b91b0
  Stored in directory: /root/.cache/pip/wheels/ee/a2/73/6f43d4f99ec69f7731617d44b68bf85aa66fc2da7b33cbe16b
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=b4c00c9dfab7ff64ef70487e328822f06f4706dbebbea9408475382c1188c189
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
Successfully built pytools pycuda


In [ ]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:TensorRT-8.2.1.8/lib

In [ ]:
!sudo apt-get install libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-dev libnvinfer-plugin8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libnvinfer8 is already the newest version (8.2.4-1+cuda11.4).
libnvinfer-dev is already the newest version (8.2.4-1+cuda11.4).
libnvinfer-plugin8 is already the newest version (8.2.4-1+cuda11.4).
libnvonnxparsers8 is already the newest version (8.2.4-1+cuda11.4).
libnvparsers8 is already the newest version (8.2.4-1+cuda11.4).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


6. Готово, вы великолепны 	☆*:.｡.o(≧▽≦)o.｡.:*☆

# Экспорт модели в TensorRT

Чтобы экспортировать модель в TensortRT, нужно сначала конвертировать ее в ONNX. Это рассматрировалось в данном [ноутбуке](https://github.com/starminalush/mlops_report/blob/main/ways_of_convert_rubert_sentiment_classification.ipynb), но на всякий случай продублируем здесь

Устанавливаем зависимости и импортируем нужные библиотеки

In [ ]:
!pip install onnx transformers onnxruntime folium==0.2.1

  Using cached onnx-1.11.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (12.8 MB)
  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
     |████████████████████████████████| 5.2 MB 3.1 MB/s 
     |████████████████████████████████| 69 kB 7.5 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
     |████████████████████████████████| 6.6 MB 35.1 MB/s 
     |████████████████████████████████| 880 kB 42.6 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=abddb9b83c70cc1298beac1673162ca79a8430ecfe65611cafb6069412d7a212
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=fa8638a69cdf393cde96a8a6ce63590894d05229c0c84bdcf1629ac21a3eec70
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad

In [42]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers.onnx import export
from pathlib import Path
from transformers.onnx import OnnxConfig
from transformers import AutoConfig
import onnxruntime as nxrun
from pathlib import Path
import onnx
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import pycuda.driver as cuda
import pycuda.autoinit
from typing import Callable, Dict, List, OrderedDict, Tuple, Mapping
import collections
import tensorrt as trt
from tensorrt import ICudaEngine, IExecutionContext
from tensorrt.tensorrt import (
    Builder,
    IBuilderConfig,
    IElementWiseLayer,
    ILayer,
    INetworkDefinition,
    IOptimizationProfile,
    IReduceLayer,
    Logger,
    OnnxParser,
    Runtime,
)
from torch.onnx import TrainingMode
import pandas as pd

Загрузим датасет для проверки качества модели

In [57]:
df = pd.read_csv('validation.csv')
df.head()

,Unnamed: 0,review,sentiment
0,0,"Очень хорошая куртка, довольно таки плотная. Я...",1
1,1,"Заказ так и не пришёл, деньги не вернули. Спор...",2
2,2,Пришло,0
3,3,"Заказывала S. Размер подошёл. Но есть одно но,...",0
4,4,на спине между лямками дырка. продавец ничего ...,0


In [58]:
texts = list(df['review'])
labels = list(df['sentiment'])

Загрузим исходную модель

In [45]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

Конвертируем модель в ONNX

In [46]:
dummy_input0 = torch.randint(1, 500, (1,512))
dummy_input1 = torch.randint(0, 1, (1,512))
dummy_input2 =  torch.randint(0, 1, (1,512))
dummy_inputs = (dummy_input0,dummy_input1,dummy_input2)
with torch.no_grad():
  symbolic_names = {0:'batch_size', 1: 'max_seq_len'} 
  torch.onnx.export(model,               
                    dummy_inputs,                        
                    "model.onnx",  
                    export_params=True,        
                    opset_version=11,          
                    do_constant_folding=True,
                    input_names = ["input_ids","attention_mask","token_type_ids"],
                    output_names = ['output'],
                    dynamic_axes={'input_ids': symbolic_names,       
                                  'attention_mask' : symbolic_names,
                                  'token_type_ids' : symbolic_names},
                    training=TrainingMode.EVAL
                    )

Загрузим модель и проверим, что все правильно, и инференс происходит без ошибок

In [47]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("model.onnx", sess_options, providers)

In [48]:
def predict_onnx(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='np')
  outputs  = model_ONNX.run(None, dict(inputs))[0][0]
  result = np.where(outputs == np.amax(outputs))[0][0]
  return result

In [49]:
text = 'Как задолбали эти тупые правила: не есть кота, не бить посуду, не есть кота'

In [50]:
predict_onnx(text)

2

Непосредственно экспорт модели в TensorRT. Код для экспорта модели был взять [отсюда](https://github.com/ELS-RD/transformer-deploy) и немного видоизменен

In [108]:
def build_engine(
    runtime: Runtime,
    onnx_file_path: str,
    logger: Logger,
    min_shape: Tuple[int, int],
    optimal_shape: Tuple[int, int],
    max_shape: Tuple[int, int],
    workspace_size: int,
) -> ICudaEngine:
    """
    Convert ONNX file to TensorRT engine.
    It supports dynamic shape, however it's advised to keep sequence length fix as it hurts performance otherwise.
    Dynamic batch size don't hurt performance and is highly advised.
    :param runtime: global variable shared accross inference call / model building
    :param onnx_file_path: path to the ONNX file
    :param logger: specific logger to TensorRT
    :param min_shape: the minimal shape of input tensors. It's advised to set first dimension (batch size) to 1
    :param optimal_shape: input tensor shape used for optimizations
    :param max_shape: maximal input tensor shape
    :param workspace_size: GPU memory to use during the building, more is always better. If there is not enough memory,
    some optimization may fail, and the whole conversion process will crash.
    :param fp16: enable FP16 precision, it usually provide a 20-30% boost compared to ONNX Runtime.
    :param int8: enable INT-8 quantization, best performance but model should have been quantized.
    :return: TensorRT engine to use during inference
    """
    with trt.Builder(logger) as builder:  # type: Builder
        with builder.create_network(
            flags=1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
        ) as network_definition:  # type: INetworkDefinition
            with trt.OnnxParser(network_definition, logger) as parser:  # type: OnnxParser
                builder.max_batch_size = max_shape[0]  # max batch size
                config: IBuilderConfig = builder.create_builder_config()
                config.max_workspace_size = workspace_size
                # to enable complete trt inspector debugging, only for TensorRT >= 8.2
                config.profiling_verbosity = trt.ProfilingVerbosity.DETAILED
                # disable CUDNN optimizations
                config.set_tactic_sources(
                    tactic_sources=1 << int(trt.TacticSource.CUBLAS) | 1 << int(trt.TacticSource.CUBLAS_LT)
                )
                config.set_flag(trt.BuilderFlag.DISABLE_TIMING_CACHE)
                # https://github.com/NVIDIA/TensorRT/issues/1196 (sometimes big diff in output when using FP16)
              
                with open(onnx_file_path, "rb") as f:
                    parsed = parser.parse(f.read())
                print(parsed)
                profile: IOptimizationProfile = builder.create_optimization_profile()
                for num_input in range(network_definition.num_inputs):
                    profile.set_shape(
                        input=network_definition.get_input(num_input).name,
                        min=min_shape,
                        opt=optimal_shape,
                        max=max_shape,
                    )
                config.add_optimization_profile(profile)
                trt_engine = builder.build_serialized_network(network_definition, config)
                engine: ICudaEngine = runtime.deserialize_cuda_engine(trt_engine)
                assert engine is not None, "error during engine generation, check error messages above :-("
                with open(trt_model_name, "wb") as f:
                  f.write(engine.serialize())

In [109]:
trt_logger: Logger = trt.Logger(trt.Logger.ERROR)
runtime: Runtime = trt.Runtime(trt_logger)
trt_model_name = "model.plan"

# create only of does not exist because it's slow to run...
if not Path(trt_model_name).exists():
    engine: ICudaEngine = build_engine(
        runtime=runtime,
        onnx_file_path="model.onnx",
        logger=trt_logger,
        min_shape=(1, 1),
        optimal_shape=(1, 128),  # num beam, batch size
        max_shape=(1, 512),  # num beam, batch size
        workspace_size=1024 * 1024**2,
    )
    

True


Теперь попробуем загрузить модель и протестировать время инференса и качество модели

In [110]:
def load_engine(runtime, engine_path):
    f = open(engine_path, 'rb')
    stream = cuda.Stream()
    engine = runtime.deserialize_cuda_engine(f.read())
    context = engine.create_execution_context()
    return context

In [111]:
model_context = load_engine(runtime, 'model.plan')

In [119]:
def predict_tensorrt(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
  with model_context as context:

     # We always use batch size 1.
    input_shape = (1, 512)
    input_nbytes = trt.volume(input_shape) * trt.int32.itemsize
    
    # Allocate device memory for inputs.
    d_inputs = [cuda.mem_alloc(input_nbytes) for binding in range(3)]
    # Create a stream in which to copy inputs/outputs and run inference.
    stream = cuda.Stream()

    # Specify input shapes. These must be within the min/max bounds of the active profile (0th profile in this case)
    # Note that input shapes can be specified on a per-inference basis, but in this case, we only have a single shape.
    for binding in range(3):
        context.set_binding_shape(binding, input_shape)
    assert context.all_binding_shapes_specified

    # Allocate output buffer by querying the size from the context. This may be different for different input shapes.
    h_output = cuda.pagelocked_empty(tuple(context.get_binding_shape(3)), dtype=np.float32)
    d_output = cuda.mem_alloc(h_output.nbytes)

    _NetworkOutput = collections.namedtuple(  # pylint: disable=invalid-name
        "NetworkOutput",
        ["output"])
    networkOutputs = []

    # Copy inputs
    input_ids = cuda.register_host_memory(np.ascontiguousarray(inputs.input_ids))
    token_type_ids = cuda.register_host_memory(np.ascontiguousarray(inputs.token_type_ids))
    attention_mask = cuda.register_host_memory(np.ascontiguousarray(inputs.attention_mask))
  
    cuda.memcpy_htod_async(d_inputs[0], input_ids, stream)
    cuda.memcpy_htod_async(d_inputs[1], token_type_ids, stream)
    cuda.memcpy_htod_async(d_inputs[2], attention_mask, stream)

    # Run inference
    context.execute_async_v2(bindings=[int(d_inp) for d_inp in d_inputs] + [int(d_output)], stream_handle=stream.handle)
    # Synchronize the stream
    stream.synchronize()
    cuda.memcpy_dtoh_async(h_output, d_output, stream)
    stream.synchronize()
    h_output = h_output[0]
    result = np.where(h_output == np.amax(h_output))[0][0]
    return result

In [121]:
%%timeit
predict_tensorrt(text)

10 loops, best of 5: 63.9 ms per loop


In [122]:
predictions = [predict_tensorrt(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.2938296903460838, recall: 0.34241900149890464, f1score: 0.17806714588197126


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


А еще можно так, но в колабе не ставится trtexec

In [ ]:
!trtexec --onnx=model.onnx --minShapes=input_ids:1x1,token_type_ids:1x1,attention_mask:1x1 --optShapes=input_ids:1x128,token_type_ids:1x128,attention_mask:1x128 --maxShapes=input_ids:1x512,token_type_ids:1x512,attention_mask:1x512 --saveEngine='model.trt'